ref: https://zenn.dev/tirimen/articles/0c70e82df4add4

ref(code): https://kiseno-log.com/2021/02/03/%E3%82%AC%E3%82%A6%E3%82%B9%E9%81%8E%E7%A8%8B%E5%9B%9E%E5%B8%B0%E3%82%92python%E3%81%A7%E5%AE%9F%E8%A3%85%E3%81%97%E3%81%A6%E3%81%BF%E3%82%8B/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [2]:
with open("p1_5S11ep=disp.txt") as f:
    text=f.readlines()
sharp_marker=[]

i_read_sharp_before=0
sharp_statements=[]
values=[]
#データをラベル情報を含む#ヘッダーの行とデータのクラスタに分離
for i in range(len(text)):
    if text[i][0]=="#":
        if i_read_sharp_before==0:
            if i!=0:
                sharp_statements.append(sharp_statement)
                values.append(value)
            sharp_statement=[]
            value=[]
        sharp_statement.append(text[i])
        i_read_sharp_before=1
    else:
        value.append(text[i])
        i_read_sharp_before=0

#データのクラスタについてさらに分離
freqs=[]
magns=[]
phass=[]
for i in range(len(values)):
    freq=[]
    magn=[]
    phas=[]
    for j in range(len(values[i])):
        val_data=values[i][j].replace("\n","").split("\t")
        freq.append(float(val_data[0]))
        magn.append(float(val_data[1]))
        phas.append(float(val_data[2]))
    freqs.append(freq)
    magns.append(magn)
    phass.append(phas)

In [3]:
height=[]
rad=[]

for j in range(len(sharp_statements)):
    value=sharp_statements[i][0][15:].split(";")
    for i in range(len(value)):
        if i == len(value)-1:
            val_label=value[i].split("=")[0]
            val=float(value[i].split("=")[1][:-2])
        else:
            val_label=value[i].split("=")[0]
            val=float(value[i].split("=")[1])
        if i==0:
            for l in range(len(freqs[j])):
                height.append(val)
        elif i==2:
            for l in range(len(freqs[j])):
                rad.append(val)

学習データを形成する。今回は周波数、膜厚、半径を用いて位相を推定するモデルを構築する。

In [4]:
freqs_flat=list(itertools.chain.from_iterable(freqs))
magns_flat=list(itertools.chain.from_iterable(magns))
phass_flat=list(itertools.chain.from_iterable(phass))

len(freqs_flat),len(magns_flat),len(phass_flat),len(height),len(rad)

(10792, 10792, 10792, 10792, 10792)

10792個のデータがあるので、個の中の8割を学習に用いて2割をテストに用いる。

In [5]:
import random

In [6]:
num_data=len(freqs_flat)
num_train=int(len(freqs_flat)*0.8)

index_list=random.sample(range(num_data-1), k=num_train)

In [7]:
freq_train=[]
magn_train=[]
phas_train=[]
heig_train=[]
radi_train=[]

freq_test=[]
magn_test=[]
phas_test=[]
heig_test=[]
radi_test=[]

for i in range(num_data):
    if i in index_list:
        freq_train.append(freqs_flat[i])
        magn_train.append(magns_flat[i])
        phas_train.append(phass_flat[i])
        heig_train.append(height[i])
        radi_train.append(rad[i])
    else:
        freq_test.append(freqs_flat[i])
        magn_test.append(magns_flat[i])
        phas_test.append(phass_flat[i])
        heig_test.append(height[i])
        radi_test.append(rad[i])


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

class kernel:
    def __init__(self, s1=1.0, s2=1.0, s3=0.1):
        self.s1, self.s2, self.s3 = s1,s2,s3

    def rbf_white(self,x, x_p,i=1):
        """カーネル計算"""
        # cdist:n-ベクトル間のユークリッド距離
        if i ==1:
            k =  self.s1 * np.exp(-1.0 * cdist(x, x_p, 'sqeuclidean')/self.s2) + (self.s3 * np.eye(x.shape[0]))
        else:
            k =  self.s1 * np.exp(-1.0 * cdist(x, x_p, 'sqeuclidean')/self.s2) 
        return k
    


# サンプルデータの作成
x_train  = [freq_train,heig_train,radi_train] # 学習データ
x_test  = [freq_test,heig_test,radi_test] # テストデータ

y_train = np.array(phas_train)

In [9]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel

regr = GaussianProcessRegressor(
    kernel=ConstantKernel(),# * RBF() + WhiteKernel(),  # カーネルの演算については後述
    alpha=0.,  # 雑音項はカーネルの中で表現しているので 0. を指定
    # optimizer=None,
)

In [10]:
len(x_train[0]),len(y_train)

(8633, 8633)

In [11]:
x=np.array(x_train[0])
x.resize(len(x),1)
y=np.array(y_train)
y.resize(len(y),1)

In [ ]:
regr.fit(x, y)
#regr.kernel_  # 推定されたカーネルと誤差項のパラメータを確認

: 

In [57]:
import GPy
import GPyOpt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
x_obs = np.asarray([[1.0, 1.5],[2.0, 0.5],[3.0, 3.5]])
y_obs = np.asarray([[4.0],[5.0],[2.0]])

bounds = [{'name': 'x1', 'type': 'continuous', 'domain': (-1.0, 5.0)},
          {'name': 'x2', 'type': 'continuous', 'domain': (-1.0, 5.0)}]

params = {'acquisition_type':'EI',#獲得関数としてEIを指定    
          'f':None, #最適化する関数の設定（実験結果は分からないので設定しない）．
          'domain':bounds, #パラメータの探索範囲の指定
          'model_type':'GP', #標準的なガウシアンプロセスを指定．
          'X':x_obs, #既知データの説明変数（インプットするx）
          'Y':y_obs, #既知データの目的変数（インプットするy）
          'de_duplication':True, #重複したデータをサンプルしないように設定．
          "normalize_Y":True, #defaltはTrue
          "exact_feval":False #defaltはFalse
          } 
myBopt = GPyOpt.methods.BayesianOptimization(**params)
myBopt.plot_acquisition()

# 次点の推奨、予測

x_suggest = myBopt.suggest_next_locations(ignored_X=x_obs) 
y_predict = myBopt.model.model.predict(x_suggest) #y_predictは(予測平均，予測分散)がタプルで返ってくる
y_mean=y_predict[0]
y_variance=y_predict[1]

print(f'つぎは{x_suggest}、yの予測平均は{y_mean}、分散は{y_variance}')

ModuleNotFoundError: No module named 'scipy.optimize'

In [27]:
def f(x):
    return np.sin(0.3 * x)
np.random.seed(42)
x_train = np.random.uniform(-10., 10., size=(10, 1))
eps_train = np.random.normal(0., 0.25, size=(10, 1))
y_train = f(x_train) + eps_train


In [30]:
y_train

array([[-0.8010237 ],
       [ 0.55914127],
       [ 0.86819758],
       [ 0.44154861],
       [-0.82038298],
       [-1.3591251 ],
       [-0.90193857],
       [ 0.66965305],
       [ 0.31694357],
       [ 1.02705196]])

In [39]:
a=np.array([2,3,4,5])
a.resize(4,1)
a

array([[2],
       [3],
       [4],
       [5]])

In [35]:
a

array([[2, 3],
       [4, 5]])